# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import geoapify_key
from api_keys import gmaps_key

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chokurdakh,70.6194,147.9024,-7.62,97,100,12.19,RU,1668892714
1,1,ushuaia,-54.8061,-68.3185,47.89,57,40,27.63,AR,1668892715
2,2,sorvag,68.0369,15.3885,41.90,50,0,6.11,NO,1668892716
3,3,barentsburg,78.0654,14.2172,38.80,97,98,10.07,SJ,1668892716
4,4,puerto ayora,-0.7472,-90.3134,81.21,96,95,7.00,EC,1668892717


---

### Create a heat map that displays the humidity for every city from Part 1:

In [25]:
# Configure the map plot

gmaps.configure(api_key= gmaps_key)
heatMapBase = gmaps.figure(center= (0, 0), zoom_level= 2)
heatMapLayer = gmaps.heatmap_layer(city_data_df[["Lat", "Lng"]], weights= city_data_df["Humidity"], max_intensity= 100, point_radius= 15)
heatMapBase.add_layer(heatMapLayer)

# Display the map
heatMapBase


Figure(layout=FigureLayout(height='420px'))

### Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
# (I set the below values to filter city_data down to 20 rows to limit api usage)

vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= 90) &
                                       (city_data_df["Max Temp"] >= 60) &
                                       (city_data_df["Humidity"] >= 50) &
                                       (city_data_df["Cloudiness"] <= 20) &
                                       (city_data_df["Wind Speed"] <= 5)                                     
                                       ]

# Drop any rows with null values
vacation_cities_df.dropna()

# Display sample data
vacation_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,mahebourg,-20.4111,57.7061,70.18,83,20,3.44,MU,1668892743
70,70,salalah,17.0175,54.1011,75.27,53,0,2.30,OM,1668892774
127,127,marzuq,14.3925,46.4701,65.80,54,6,1.63,YE,1668892825
201,201,ijaki,22.9372,57.7571,69.80,53,0,3.00,OM,1668892889
202,202,bambous virieux,-20.3472,57.7647,70.36,83,20,3.44,MU,1668892890


### Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
35,mahebourg,MU,-20.4111,57.7061,83,
70,salalah,OM,17.0175,54.1011,53,
127,marzuq,YE,14.3925,46.4701,54,
201,ijaki,OM,22.9372,57.7571,53,
202,bambous virieux,MU,-20.3472,57.7647,83,


### Use Google Places API to find the first hotel for each city located within 5,000 meters of your coordinates.

In [42]:
#####
# Note, per the documentation at: https://developers.google.com/maps/documentation/places/web-service/search-nearby
# 
# "When using rankby=distance, # the radius parameter will not be accepted, # and will result in an INVALID_REQUEST."
# 
# However, rankby=distance is the best option we have for easily finding the closest hotel
# Therefore, we will search within the default 50,000 meters instead to avoid an error
#
#####

# Set parameters to search for a hotel

params = {
    "type": "lodging",
    "rankby": "distance",
    "key": gmaps_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
   
    # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params= params).json()
        
    # extract results
    results = response['results']
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
mahebourg - nearest hotel: Residence Les Bambous
salalah - nearest hotel: Start Emirates نجم الامارات للشق المفروشه
marzuq - nearest hotel: حيد القفلة
ijaki - nearest hotel: سالم بن عبدالله بن سالم القصابي
bambous virieux - nearest hotel: Domaine de chasseur
tsihombe - nearest hotel: Imongy II Rebeke
port blair - nearest hotel: Pink Fly Lounge
grand river south east - nearest hotel: Seaview Villa
ocos - nearest hotel: Cupido
cayenne - nearest hotel: Hôtel Ket Tai
namikupa - nearest hotel: The Real Life Hotel
sambava - nearest hotel: Hotel rija
gushikawa - nearest hotel: Smilax Kurichi
kavaratti - nearest hotel: Poddar Sworone
xinpu - nearest hotel: 遠東新埔廠禮堂
antalaha - nearest hotel: Hotel Nanie
niamtougou - nearest hotel: Sevebat
eureka - nearest hotel: Abigail's Elegant Victorian Mansion
alofi - nearest hotel: Bella's Guesthouse
lahij - nearest hotel: YAFA


,City,Country,Lat,Lng,Humidity,Hotel Name
35,mahebourg,MU,-20.4111,57.7061,83,Residence Les Bambous
70,salalah,OM,17.0175,54.1011,53,Start Emirates نجم الامارات للشق المفروشه
127,marzuq,YE,14.3925,46.4701,54,حيد القفلة
201,ijaki,OM,22.9372,57.7571,53,سالم بن عبدالله بن سالم القصابي
202,bambous virieux,MU,-20.3472,57.7647,83,Domaine de chasseur


### Plot the hotels on top of the humidity heatmap, with each pin containing the Hotel Name, City, and Country

In [43]:
# Create a template for the hover messages (in this case, click messages)
info_box_template = """
<dl>
<dt>Name</dt>
<dd>{Hotel Name}</dd>
<dt>City</dt>
<dd>{City}</dd>
<dt>Country</dt>
<dd>{Country}</dd>
</dl>
"""

# Configure the map plot
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(0, 0), zoom_level= 2)

# Add the humidity layer
heat_layer = gmaps.heatmap_layer(city_data_df[["Lat", "Lng"]], weights= city_data_df["Humidity"], dissipating= True, max_intensity= 100, point_radius= 15)

# Add the hotel markers layer
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

# Display the map
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))